# Convolutional neural network

In [1]:
using MLDatasets
using Flux
using Flux: @epochs, onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated, partition
# using CuArrays

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #RegisterDataDep#31(::Function, ::Function, ::Function, ::String, ::String, ::String, ::String) at ./deprecated.jl:57
 [3] (::DataDeps.#kw##RegisterDataDep)(::Array{Any,1}, ::DataDeps.#RegisterDataDep, ::String, ::String, ::String, ::String) at ./<missing>:0
 [4] __init__() at /home/pika/.julia/v0.6/MLDatasets/src/CIFAR10/CIFAR10.jl:60
 [5] _require_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:200
 [6] _require_search_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:236
 [7] _require(::Symbol) at ./loading.jl:441
 [8] require(::Symbol) at ./loading.jl:405
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/pika/.julia/v0.6/IJulia/src/execute_request.jl:158
 [10] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/pika/.julia/v0.6/Compat/src/Compat.jl:378
 [11] eventloop(::ZMQ.Socket) at /home/pika/.julia/v0.6/IJulia/src

## Load data

Classify MNIST digits with a convolutional network

In [2]:
train_x, train_y = CIFAR10.traindata()
test_x,  test_y  = CIFAR10.testdata()

(FixedPointNumbers.Normed{UInt8,8}[0.62N0f8 0.596N0f8 … 0.239N0f8 0.212N0f8; 0.624N0f8 0.592N0f8 … 0.192N0f8 0.22N0f8; … ; 0.494N0f8 0.49N0f8 … 0.114N0f8 0.133N0f8; 0.455N0f8 0.467N0f8 … 0.078N0f8 0.082N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.439N0f8 0.439N0f8 … 0.455N0f8 0.42N0f8; 0.435N0f8 0.431N0f8 … 0.4N0f8 0.412N0f8; … ; 0.357N0f8 0.357N0f8 … 0.322N0f8 0.329N0f8; 0.333N0f8 0.345N0f8 … 0.251N0f8 0.263N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.192N0f8 0.2N0f8 … 0.659N0f8 0.627N0f8; 0.184N0f8 0.157N0f8 … 0.58N0f8 0.584N0f8; … ; 0.141N0f8 0.125N0f8 … 0.494N0f8 0.506N0f8; 0.129N0f8 0.133N0f8 … 0.42N0f8 0.431N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.922N0f8 0.933N0f8 … 0.322N0f8 0.333N0f8; 0.906N0f8 0.922N0f8 … 0.18N0f8 0.243N0f8; … ; 0.914N0f8 0.925N0f8 … 0.725N0f8 0.706N0f8; 0.91N0f8 0.922N0f8 … 0.733N0f8 0.729N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.922N0f8 0.933N0f8 … 0.376N0f8 0.396N0f8; 0.906N0f8 0.922N0f8 … 0.224N0f8 0.294N0f8; … ; 0.914N0f8 0.925N0f8 … 0.784N0f8 0.765N0f8; 

In [3]:
train_x = Float64.(train_x / 255)
train_y = onehotbatch(train_y, 0:9)
test_x = Float64.(test_x / 255)
test_y = onehotbatch(test_y, 0:9)

10×10000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
  true  false  false  false  false     false   true  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false   true  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false     false  false  false  false   true
 false   true   true  false  false      true  false  false  false  false
 false  false  false  false  false     false  false  false  false  false

## Preprocessing

In [4]:
# Partition into batches of size 1,000
train = [(train_x[:, :, :, i:(i+999)], train_y[:, i:(i+999)]) for i = 1:1000:50_000];
# train = gpu.(train)

In [5]:
# Prepare test set (first 1,000 images)
test_x = test_x[:, :, :, 1:1000]# |> gpu
test_y = test_y[:, 1:1000]# |> gpu

10×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false  false  false  false     false  false   true  false  false
 false  false  false  false  false     false  false  false  false  false
  true  false  false  false  false      true  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false   true   true  false  false     false   true  false  false   true
 false  false  false  false  false     false  false  false  false  false

## Model

In [6]:
m = Chain(
    Conv((3, 3), 3=>32, relu),  # (32, 32, 3) -> (30, 30, 32)
    Conv((3, 3), 32=>64, relu),  # (30, 30, 32) -> (28, 28, 64)
    x -> maxpool(x, (2, 2); stride=2),  # (28, 28, 64) -> (14, 14, 64)
    BatchNorm(64),
#     Dropout(0.25),
    Conv((3, 3), 64=>128, relu),  # (14, 14, 64) -> (12, 12, 128)
    x -> maxpool(x, (2, 2); stride=2),  # (12, 12, 128) -> (6, 6, 128)
    Conv((2, 2), 128=>128, relu),  # (6, 6, 128) -> (5, 5, 128)
    x -> maxpool(x, (2, 2); stride=2),  # (5, 5, 128) -> (2, 2, 128)
    BatchNorm(128),
#     Dropout(0.25),
    x -> reshape(x, :, size(x, 4)),  # (2, 2, 128) -> 512
    Dense(512, 1500, relu),
    BatchNorm(1500),
#     Dropout(0.5),
    Dense(1500, 10), softmax)# |> gpu

Chain(Conv((3, 3), 3=>32, NNlib.relu), Conv((3, 3), 32=>64, NNlib.relu), #3, BatchNorm(64), Conv((3, 3), 64=>128, NNlib.relu), #4, Conv((2, 2), 128=>128, NNlib.relu), #5, BatchNorm(128), #6, Dense(512, 1500, NNlib.relu), BatchNorm(1500), Dense(1500, 10), NNlib.softmax)

In [7]:
m(train[1][1])  # test if model works

Tracked 10×1000 Array{Float64,2}:
 0.0738703   0.226183   0.104323     …  0.304167   0.0051662   0.00391455
 0.0146821   0.0292849  0.0292691       0.0479552  0.0820759   0.0418958 
 0.00646479  0.0185909  0.0424737       0.0133645  0.281906    0.1245    
 0.0666114   0.160932   0.0808966       0.0631955  0.00928798  0.0262703 
 0.0136798   0.105351   0.0144587       0.101268   0.0540482   0.14778   
 0.127966    0.134166   0.575405     …  0.15226    0.112548    0.0525409 
 0.0632751   0.208754   0.0994077       0.0126038  0.380322    0.00624554
 0.481846    0.0219483  0.0316484       0.0852076  0.00545263  0.298559  
 0.0402008   0.0564432  0.000262631     0.156596   0.0500804   0.0139881 
 0.111404    0.0383461  0.0218554       0.0633829  0.0191137   0.284306  

## Loss function

In [8]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

In [9]:
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

accuracy (generic function with 1 method)

## Optimizer

In [10]:
evalcb = throttle(() -> @show(accuracy(test_x, test_y)), 10)
opt = ADAM(params(m))

(::#80) (generic function with 1 method)

## Training

In [11]:
@epochs 5 Flux.train!(loss, train, opt, cb=evalcb)

INFO: Epoch 1


accuracy(test_x, test_y) = 0.188
accuracy(test_x, test_y) = 0.285
accuracy(test_x, test_y) = 0.299
accuracy(test_x, test_y) = 0.349
accuracy(test_x, test_y) = 0.353
accuracy(test_x, test_y) = 0.366
accuracy(test_x, test_y) = 0.4
accuracy(test_x, test_y) = 0.396
accuracy(test_x, test_y) = 0.455
accuracy(test_x, test_y) = 0.424
accuracy(test_x, test_y) = 0.439
accuracy(test_x, test_y) = 0.425
accuracy(test_x, test_y) = 0.454
accuracy(test_x, test_y) = 0.46
accuracy(test_x, test_y) = 0.445
accuracy(test_x, test_y) = 0.493
accuracy(test_x, test_y) = 0.488


INFO: Epoch 2


accuracy(test_x, test_y) = 0.491
accuracy(test_x, test_y) = 0.512
accuracy(test_x, test_y) = 0.496
accuracy(test_x, test_y) = 0.472
accuracy(test_x, test_y) = 0.483
accuracy(test_x, test_y) = 0.491
accuracy(test_x, test_y) = 0.492
accuracy(test_x, test_y) = 0.486
accuracy(test_x, test_y) = 0.519
accuracy(test_x, test_y) = 0.51
accuracy(test_x, test_y) = 0.512
accuracy(test_x, test_y) = 0.52
accuracy(test_x, test_y) = 0.533
accuracy(test_x, test_y) = 0.533
accuracy(test_x, test_y) = 0.519
accuracy(test_x, test_y) = 0.55
accuracy(test_x, test_y) = 0.551


INFO: Epoch 3


accuracy(test_x, test_y) = 0.538
accuracy(test_x, test_y) = 0.552
accuracy(test_x, test_y) = 0.535
accuracy(test_x, test_y) = 0.533
accuracy(test_x, test_y) = 0.535
accuracy(test_x, test_y) = 0.54
accuracy(test_x, test_y) = 0.558
accuracy(test_x, test_y) = 0.538
accuracy(test_x, test_y) = 0.558
accuracy(test_x, test_y) = 0.562
accuracy(test_x, test_y) = 0.56
accuracy(test_x, test_y) = 0.554


LoadError: [91mInterruptException:[39m